## CVE Explore

In [1]:
import json
import pandas as pd
#import urllib.request
#import xml.etree.ElementTree as ET
#import xmltodict
#import spacy
import numpy as np
import nltk
#import itertools
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

## Load CVE Data

In [2]:
with open('Data/all_CVE_data.json') as f:
    cve_dict = json.load(f)

## Get CVE Descriptions

In [3]:
cve_desc_list = []

# About 202,000 Items
file_name = 'Data/all_CVE_descriptions.txt'
with open(file_name, 'w') as f:
    for i in range(len(cve_dict['cve']['item'])):
        cve_desc_list.append(cve_dict['cve']['item'][i]['desc'])
        f.write(cve_dict['cve']['item'][i]['desc'] +'\n')
f.close()

## Generate corpus

In [4]:
# Create corpus from CVE descriptions
corpus = ''

for desc in cve_desc_list:
    try:
        corpus += desc + ' '
    except:
        if desc:
            corpus += desc['html:p'][0]

## Remove stopwords, punctuation, and make lower case

In [5]:
stop_words = set(stopwords.words('english'))  

word_tokens = word_tokenize(corpus.lower())
  
filtered_corpus = [w for w in word_tokens if not w in stop_words]  
  
filtered_corpus = []  
  
for w in word_tokens:  
    if w not in stop_words and w.isalpha():  
        filtered_corpus.append(w)  

## Function to make co-occurrence matrix

In [6]:
def generate_co_occurrence_matrix(corpus,window):
    vocab = set(corpus)
    vocab = list(vocab)
    vocab_index = {word: i for i, word in enumerate(vocab)}
    
    # Ivan - Modification - Window N
    print('[START] Create word pairs')
    N_grams = []
    for i in range(len(corpus)):
        for j in range(1,window+1):
            if i+j<len(corpus):
                N_grams.append((corpus[i],corpus[i+j]))
            if i-j>=0:
                N_grams.append((corpus[i],corpus[i-j]))
    print('[COMPLETE] Create word pairs')
    # Frequency distribution of bigrams ((word1, word2), num_occurrences)
    print('[START] Calculate number of occurrences')
    ngram_freq = nltk.FreqDist(N_grams).most_common(len(N_grams))
    print('[COMPLETE] Calculate number of occurrences')
    # Initialise co-occurrence matrix
    # co_occurrence_matrix[current][previous]
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))
 
    # Loop through the bigrams taking the current and previous word,
    # and the number of occurrences of the bigram.
    print('[START] Generate Co-occurrence Matrix')
    for ngram in ngram_freq:
        current = ngram[0][1]
        previous = ngram[0][0]
        count = ngram[1]
        pos_current = vocab_index[current]
        pos_previous = vocab_index[previous]
        co_occurrence_matrix[pos_current][pos_previous] = count
    co_occurrence_matrix = np.matrix(co_occurrence_matrix)
    print('[COMPLETE] Generate Co-occurrence Matrix')
    
    # return the matrix and the index
    return co_occurrence_matrix, vocab_index

## Generate co-occurrence matrix - Window set to 3

In [7]:
matrix, vocab_index = generate_co_occurrence_matrix(filtered_corpus,5)

[START] Create word pairs
[COMPLETE] Create word pairs
[START] Calculate number of occurrences
[COMPLETE] Calculate number of occurrences
[START] Generate Co-occurrence Matrix
[COMPLETE] Generate Co-occurrence Matrix


In [8]:
data_matrix = pd.DataFrame(matrix, index=vocab_index,
                             columns=vocab_index)

## Sort by the sum of the all columns

In [9]:
#Sort rows of data_matrix by the sum of the columns of each row in descending order
data_matrix['sum_column'] = data_matrix.sum(axis=1)
data_matrix = data_matrix.sort_values(by='sum_column', ascending=False)
data_matrix = data_matrix.drop(columns=['sum_column'])

## Sort by the sum of all rows

In [10]:
data_matrix.loc['sum_row'] = data_matrix.sum(axis=1)
data_matrix = data_matrix.sort_values(by = 'sum_row',ascending=False, axis=1)
data_matrix = data_matrix.drop(index='sum_row')

In [11]:
data_matrix

,candidate,reserved,vulnerability,allows,via,remote,attackers,use,arbitrary,security,...,addupdate,dominic,dbimagegallery,fmm,ascom,aw,cdwidgetid,wayland,fingerprintdo,iproom
candidate,157400.0,121903.0,6252.0,344.0,220.0,183.0,30.0,9360.0,105.0,44302.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
reserved,121903.0,88244.0,1171.0,45.0,1182.0,76.0,30.0,44165.0,193.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vulnerability,6252.0,1171.0,6888.0,13652.0,901.0,9274.0,1581.0,1182.0,786.0,1621.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
allows,344.0,45.0,13652.0,96.0,1107.0,56887.0,57450.0,330.0,3422.0,489.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
via,220.0,1182.0,901.0,1107.0,224.0,518.0,2878.0,244.0,26046.0,186.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
aw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cdwidgetid,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wayland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fingerprintdo,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 500

In [22]:
df = data_matrix.iloc[0:50,0:50]
plt.figure(figsize=(25,25))
plt.imshow(df)
plt.colorbar()
plt.xticks(range(len(df)),df.columns, rotation=90)
plt.yticks(range(len(df)),df.index)
plt.tick_params(axis='both', which='major', labelsize=10, labelbottom = False, bottom=False, top = False, labeltop=True)
plt.show()

## Save Sorted Data Matrix to CSV

In [14]:
data_matrix.to_csv('Data/CVE_coocurrence_matrix.csv')

KeyboardInterrupt: 